In [ ]:
# $ pip install cryptocmd # need installation firstly
from cryptocmd import CmcScraper
import requests
import re
import time
import datetime
import pandas as pd
import numpy as np
import json
import tushare as ts

In [ ]:
def get_data(date):
    url1 = "https://min-api.cryptocompare.com/data/histohour?fsym=XRP&tsym=USD&limit=2000&toTs={}".format(date)
    r = requests.get(url1)
    ipdata = r.json()
    return ipdata

In [ ]:
def get_df(from_date, to_date):
    date = to_date
    holder = []
    # While the earliest date returned is later than the earliest date requested, keep on querying the API
    # and adding the results to a list.
    while date > from_date:
        data = get_data(date)
        holder.append(pd.DataFrame(data['Data']))
        date = data['TimeFrom']
        # Join together all of the API queries in the list.
        df = pd.concat(holder, axis = 0)
        # Remove data points from before from_date
        df = df[df['time']>from_date]
        # Convert to timestamp to readable date format
        df['time'] = pd.to_datetime(df['time'], unit='s')
        # Make the DataFrame index the time
        df.set_index('time', inplace=True)
        # And sort it so its in time order
        df.sort_index(ascending=False, inplace=True)
    return df

In [ ]:
#for hourly data #from the website of cryptocompare
holder = []
from_date = 1552431600 #2019.3.13 0:00
to_date = 1560380400 # 2019.6.12 23:00


holder.append(get_df(from_date, to_date))
df = pd.concat(holder, axis = 1)
df=df.sort_index()
df.to_csv("/Users/yitingjiang/Desktop/df.csv")

In [ ]:
#for daily data #from the website of coinmarketcap

# initialise scraper
scraper = CmcScraper('XRP', '2-12-2018', '12-6-2019')

# get data as list of list
headers, data = scraper.get_data()

xrp_json_data=scraper.get_data('json')

# export the data to csv
scraper.export('csv',name='ripple')

# get dataframe for the data
dff = scraper.get_dataframe()